In [ ]:
import numpy as np
from collections import Counter # Dont forget to import this dependency

class KNearestNeighbors:
    """
    K-Nearest Neighbors (KNN) from scratch.
    Supports classification and regression.
    """

    def __init__(self, k=3, mode="classification", distance="euclidean"):
        """
        Parameters
        ----------
        k : int
            Number of neighbors to consider.
        mode : str, "classification" or "regression"
            Determines prediction type.
        distance : str, "euclidean" or "manhattan"
            Distance metric.
        """
        self.k = k
        self.mode = mode
        self.distance = distance
        self.X_train = None
        self.y_train = None

    def _compute_distances(self, X):
        """Compute pairwise distances between X and stored training data."""
        if self.distance == "euclidean":
            # Efficient vectorized Euclidean distance
            # (a-b)^2 = a^2 + b^2 - 2ab
            X2 = np.sum(X**2, axis=1, keepdims=True)
            X_train2 = np.sum(self.X_train**2, axis=1)
            cross = X @ self.X_train.T
            dists = np.sqrt(X2 + X_train2 - 2 * cross)
        elif self.distance == "manhattan":
            # Broadcasting trick for L1 distance
            dists = np.sum(np.abs(X[:, np.newaxis] - self.X_train), axis=2)
        else:
            raise ValueError("Unsupported distance metric")
        return dists

    def fit(self, X, y):
        """Store training data."""
        self.X_train = np.asarray(X, dtype=np.float64)
        self.y_train = np.asarray(y)

    def predict(self, X):
        """Predict labels for given data."""
        X = np.asarray(X, dtype=np.float64)
        dists = self._compute_distances(X)

        # Find k nearest indices
        neighbors_idx = np.argsort(dists, axis=1)[:, :self.k]
        neighbors_labels = self.y_train[neighbors_idx]

        if self.mode == "classification":
            # Majority vote
            preds = []
            for row in neighbors_labels:
                most_common = Counter(row).most_common(1)[0][0] # Each row is a list/array of labels of the KNN, Counter function creates a dictionary
                # of the labels:count , and most_common returns a tuple of (label,count), since it's (1)-> it returns a single tuple and first [0]
                # extracts that tuple and second [0] extracts the first value from that tuple which is the label. 
                preds.append(most_common)
            return np.array(preds)
        elif self.mode == "regression":
            # Mean of neighbors
            return np.mean(neighbors_labels, axis=1)
        else:
            raise ValueError("Unsupported mode")

    def score(self, X, y):
        """Evaluate model performance."""
        preds = self.predict(X)
        if self.mode == "classification":
            return np.mean(preds == y)
        elif self.mode == "regression":
            return np.mean((preds - y) ** 2)  # MSE
        
np.random.seed(42)
X_train = np.random.rand(10, 2)
y_train = np.random.choice([0, 1], size=10)

knn = KNearestNeighbors(k=3, mode="classification")
knn.fit(X_train,y_train)

X_test = np.random.rand(5, 2)
print("Predictions:", knn.predict(X_test))

y_train_reg = np.random.rand(10)
knn = KNearestNeighbors(k=3,mode="regression")
knn.fit(X_train,y_train_reg)
print("Predictions (regression):", knn.predict(X_test))        

In [4]:
import numpy as np
from collections import Counter

🔹 What X2 = np.sum(X**2, axis=1, keepdims=True) really means

X: shape (m_test, d) → m_test points, d features each.

X**2: square each element → shape (m_test, d).

np.sum(..., axis=1, keepdims=True) → sum across features → shape (m_test, 1).

So for each test sample, you’re computing:

𝑋
2
[
𝑖
]
=
∑
𝑘
=
1
𝑑
(
𝑥
𝑖
,
𝑘
)
2
X2[i]=
k=1
∑
d
	​

(x
i,k
	​

)
2

👉 This is the squared norm (‖x‖²) of each row vector, not just squaring elements individually.

🔹 Why do we need the sum?

Because of the Euclidean distance formula:

∥
𝑥
−
𝑦
∥
2
=
∥
𝑥
∥
2
+
∥
𝑦
∥
2
−
2
(
𝑥
⋅
𝑦
)
∥x−y∥
2
=∥x∥
2
+∥y∥
2
−2(x⋅y)

∥
𝑥
∥
2
=
∑
𝑘
𝑥
𝑘
2
∥x∥
2
=∑
k
	​

x
k
2
	​

 → that’s why we take the sum.

If we only did X**2, we’d have each squared coordinate, not the total squared length.

🔹 Shape intuition

Say X = [[1,2], [3,4]] → shape (2,2)

X**2 = [[1,4], [9,16]]

np.sum(..., axis=1, keepdims=True) =
[[1+4], [9+16]] = [[5], [25]] → squared norms.

That way:

X2: shape (m_test,1)

X_train2: shape (m_train,)

Combine them with broadcasting in the formula.

🎯 Intuition

We don’t just want to square elements; we want the sum of squares per vector (per row) because that’s exactly the squared length used in the Euclidean distance trick.

In [ ]:
import numpy as np
from collections import Counter # Dont forget to import this dependency

class KNearestNeighbors:
    """
    K-Nearest Neighbors (KNN) from scratch.
    Supports classification and regression.
    """

    def __init__(self, k=3, mode="classification", distance="euclidean"):
        """
        Parameters
        ----------
        k : int
            Number of neighbors to consider.
        mode : str, "classification" or "regression"
            Determines prediction type.
        distance : str, "euclidean" or "manhattan"
            Distance metric.
        """
        self.k = k
        self.mode = mode
        self.distance = distance
        self.X_train = None
        self.y_train = None

    def _compute_distances(self, X):
        """Compute pairwise distances between X and stored training data."""
        if self.distance == "euclidean":
            # Efficient vectorized Euclidean distance
            # (a-b)^2 = a^2 + b^2 - 2ab
            X2 = np.sum(X**2, axis=1, keepdims=True)
            X_train2 = np.sum(self.X_train**2, axis=1)
            cross = X @ self.X_train.T
            dists = np.sqrt(X2 + X_train2 - 2 * cross)
        elif self.distance == "manhattan":
            # Broadcasting trick for L1 distance
            dists = np.sum(np.abs(X[:, np.newaxis] - self.X_train), axis=2)
        else:
            raise ValueError("Unsupported distance metric")
        return dists

    def fit(self, X, y):
        """Store training data."""
        self.X_train = np.asarray(X, dtype=np.float64)
        self.y_train = np.asarray(y)

    def predict(self, X):
        """Predict labels for given data."""
        X = np.asarray(X, dtype=np.float64)
        dists = self._compute_distances(X)

        # Find k nearest indices
        neighbors_idx = np.argsort(dists, axis=1)[:, :self.k]
        neighbors_labels = self.y_train[neighbors_idx]

        if self.mode == "classification":
            # Majority vote
            preds = []
            for row in neighbors_labels:
                most_common = Counter(row).most_common(1)[0][0] # Each row is a list/array of labels of the KNN, Counter function creates a dictionary
                # of the labels:count , and most_common returns a tuple of (label,count), since it's (1)-> it returns a single tuple and first [0]
                # extracts that tuple and second [0] extracts the first value from that tuple which is the label. 
                preds.append(most_common)
            return np.array(preds)
        elif self.mode == "regression":
            # Mean of neighbors
            return np.mean(neighbors_labels, axis=1)
        else:
            raise ValueError("Unsupported mode")

    def score(self, X, y):
        """Evaluate model performance."""
        preds = self.predict(X)
        if self.mode == "classification":
            return np.mean(preds == y)
        elif self.mode == "regression":
            return np.mean((preds - y) ** 2)  # MSE
        
np.random.seed(42)
X_train = np.random.rand(10, 2)
y_train = np.random.choice([0, 1], size=10)

knn = KNearestNeighbors(k=3, mode="classification")
knn.fit(X_train,y_train)

X_test = np.random.rand(5, 2)
print("Predictions:", knn.predict(X_test))

y_train_reg = np.random.rand(10)
knn = KNearestNeighbors(k=3,mode="regression")
knn.fit(X_train,y_train_reg)
print("Predictions (regression):", knn.predict(X_test))        

In [9]:
np.random.seed(42)
X_train = np.random.rand(10, 2)
y_train = np.random.choice([0, 1], size=10)

knn = KNearestNeighbors(k=3, mode="classification")
knn.fit(X_train,y_train)

X_test = np.random.rand(5, 2)
print("Predictions:", knn.predict(X_test))

y_train_reg = np.random.rand(10)
knn = KNearestNeighbors(k=3,mode="regression")
knn.fit(X_train,y_train_reg)
print("Predictions (regression):", knn.predict(X_test))

Predictions: [1 1 1 1 0]
Predictions (regression): [0.35285689 0.55136222 0.57186389 0.13028021 0.5177212 ]


In [ ]:
import numpy as np
from collections import Counter

class KNearest_Neighbors:
    def __init__(self, k=1, mode="classification", distance="euclidean"):
        self.k = k
        self.mode = mode
        self.distance = distance

        self.X_train = None
        self.y_train = None

    def fit(self,X,y):
        X = np.asarray(X,dtype=np.float64)
        Y = np.asarray(y)
        self.X_train = X
        self.y_train = Y

    def compute_distances(self,X):    
        if self.distance == "euclidean":
            X2 = np.sum(X ** 2, axis=1, keepdims=True)
            X_train2 = np.sum(self.X_train ** 2, axis=1)
            cross = X @ self.X_train.T
            distance = np.sqrt(X2 + X_train2 - 2 * cross)
            return distance
        elif self.distance == "manhattan":
            return np.sum(np.abs(X[:,np.newaxis] - self.X_train), axis=2)
        else:
            raise ValueError("Unsupported distance metric")    
        

    def predict(self,X):
        X = np.asarray(X,np.float64)
        distances = self.compute_distances(X) 

        neighbor_idx = np.argsort(distances, axis=1)[:,:self.k]   
        labels = self.y_train[neighbor_idx]

        if self.mode == "classification":
            preds = []
            for row in labels:
                most_common = Counter(row).most_common(1)[0][0]
                preds.append(most_common)
            return preds
        elif self.mode == "regression":
            return np.mean(labels,axis=1)
        else:
            raise ValueError("Unsupported mode")   

    def score(self,X,y):
        y_preds = self.predict(X)
        if self.mode == "classification":
            return np.mean(y_preds == y)
        else:
            return np.mean((y_preds - y) ** 2)     

In [8]:
np.random.seed(42)
X = np.random.randn(10,2)
Y = np.random.choice(2,10)

model = KNearest_Neighbors(k=3,distance="euclidean")
model.fit(X,Y)

X_test = np.random.randn(5,2)
Y_test = np.random.choice(2,5)
print(Y_test)
print(model.predict(X_test))

y_train_reg = np.random.rand(10)
y_test_reg = np.random.rand(5)
print(y_test_reg)
model.fit(X,y_train_reg)
print(model.predict(X_test))
model.score(X_test,y_test_reg)

[1 0 1 1 1]
[np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0)]
[0.32654077 0.57044397 0.52083426 0.96117202 0.84453385]
[np.float64(0.20794166286818883), np.float64(0.03131329245555858), np.float64(0.7553614103176525), np.float64(0.9266588657937942), np.float64(0.20794166286818883)]


np.float64(0.0)